In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import scipy.stats as stats
import wandb

In [2]:
api = wandb.Api()
run = api.run("dgcnz/PARTv2-classification-ft/djrrifsi")
df = run.history()

In [3]:
df.columns

Index(['_step', 'val/acc5', '_runtime', 'val/loss', 'train/loss', 'epoch',
       'train/lr', 'val/acc1', 'max_accuracy', '_timestamp'],
      dtype='object')

In [4]:
# Define custom functions
def log_func(x, a, b):
    return a * np.log(x) + b

def log_linear_func(x, a, b, c):
    return a * np.log(x) + b * x + c

def power_func(x, a, b, c):
    return a - b * x**(-c)

def rational_func(x, a, b, c):
    # Monotonic function that starts linear and saturates
    return a + (b * x) / (c + x)

def generalized_log(x, a, b, c, d):
    # Monotonic function that introduces an extra curvature parameter d
    return a + b * (np.log(1 + c * x))**d

def modified_log_linear(x, a, b, c):
    # Monotonic and log-linear-like behavior.
    return a + b * np.log(1 + c * x)

In [5]:
colors = {
    "red": '#%02x%02x%02x' % (255, 59, 48),
    "orange": '#%02x%02x%02x' % (255, 149, 0),
    "yellow": '#%02x%02x%02x' % (255, 204, 0),
    "green": '#%02x%02x%02x' % (40, 205, 65),
    "mint": '#%02x%02x%02x' % (0, 199, 190),
    "teal": '#%02x%02x%02x' % (89, 173, 196),
    "cyan": '#%02x%02x%02x' % (85, 190, 240),
    "blue": '#%02x%02x%02x' % (0, 122, 255),
    "indigo": '#%02x%02x%02x' % (88, 86, 214),
    "purple": '#%02x%02x%02x' % (175, 82, 222),
    "pink": '#%02x%02x%02x' % (255, 45, 85),
    "brown": '#%02x%02x%02x' % (162, 132, 94),
    "gray": '#%02x%02x%02x' % (142, 142, 147),
    "black": '#%02x%02x%02x' % (0, 0, 0),
}

In [16]:
# Extract step and bbox/AP columns
steps = df['epoch'].values
acc_scores = df['val/acc1'].values

n = len(steps)
weights = np.linspace(1, 10, n)
sigma = 1 / weights

# Fit the functions
popt_log, _ = curve_fit(log_func, steps[1:], acc_scores[1:], sigma=sigma[1:])
popt_log_linear, _ = curve_fit(log_linear_func, steps[1:], acc_scores[1:], sigma=sigma[1:])
popt_power, _ = curve_fit(power_func, steps[1:], acc_scores[1:], p0=[50, 50, 0.1], sigma=sigma[1:])
# popt_rational, _ = curve_fit(rational_func, steps, acc_scores, p0=[50, 50, 0.1], sigma=sigma)
# popt_generalized_log, _ = curve_fit(generalized_log, steps, acc_scores, p0=[50, 50, 0.1, 1])
# popt_modified_log_linear, _ = curve_fit(modified_log_linear, steps, acc_scores, p0=[50, 50, 0.1], maxfev=5000)

print("params", popt_log_linear)


# Generate points for the fitted curves
x_fit = np.arange(steps[1], 310, 1)
y_fit_log = log_func(x_fit, *popt_log)
print(x_fit)
y_fit_log_linear = log_linear_func(x_fit, *popt_log_linear)
y_fit_power = power_func(x_fit, *popt_power)
# y_fit_rational = rational_func(x_fit, *popt_rational)
# y_fit_generalized_log = generalized_log(x_fit, *popt_generalized_log)
# y_fit_modified_log_linear = modified_log_linear(x_fit, *popt_modified_log_linear)

# Create forecasts
future_steps = np.array([70, 80, 90, 100, 110, 300, 330])
log_forecast = log_func(future_steps, *popt_log)
log_linear_forecast = log_linear_func(future_steps, *popt_log_linear)
power_forecast = power_func(future_steps, *popt_power)
# rational_forecast = rational_func(future_steps, *popt_rational)
# generalized_log_forecast = generalized_log(future_steps, *popt_generalized_log)
# modified_log_linear_forecast = modified_log_linear(future_steps, *popt_modified_log_linear)

fig = px.scatter(df, x="epoch", y="val/acc1")

# Update layout
fig.update_layout(
    title='Image Classification Model Performance with different model fits',
    xaxis_title='Epoch',
    yaxis_title='Accuracy',
    legend_title='Legend',
    hovermode="x"
)
print("x_fit", x_fit)
print("y_fit_log", y_fit_log)
fig.add_scatter(x=x_fit, y=y_fit_log, mode='lines', name='Logarithmic fit')
# fig.add_scatter(x=x_fit, y=y_fit_log_linear, mode='lines', name='Log-Linear fit')
fig.add_scatter(x=x_fit, y=y_fit_power, mode='lines', name='Power law fit')
# fig.add_scatter(x=x_fit, y=y_fit_rational, mode='lines', name='Rational fit')
# fig.add_scatter(x=x_fit, y=y_fit_generalized_log, mode='lines', name='Generalized Log fit')
# fig.add_scatter(x=x_fit, y=y_fit_modified_log_linear, mode='lines', name='Modified Log-Linear fit')
fig.show()

# Print forecasts
print("Logarithmic Forecast:")
for step, forecast in zip(future_steps, log_forecast):
    print(f"Step {step}: {forecast:.4f}")

print("\nLog-Linear Forecast:")
for step, forecast in zip(future_steps, log_linear_forecast):
    print(f"Step {step}: {forecast:.4f}")

print("\nPower law Forecast:")
for step, forecast in zip(future_steps, power_forecast):
    print(f"Step {step}: {forecast:.4f}")

plt.show()

# Calculate Mean Squared Error for each model
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

mse_log = mse(acc_scores[1:], log_func(steps[1:], *popt_log))
mse_log_linear = mse(acc_scores[1:], log_linear_func(steps[1:], *popt_log_linear))
mse_power = mse(acc_scores[1:], power_func(steps[1:], *popt_power))


print("\nMean Squared Error:")
print(f"Logarithmic: {mse_log:.6f}")
print(f"Power law: {mse_power:.6f}")
print(f"Log linear: {mse_log_linear:.6f}")

params [10.12955676 -0.04587898 40.04710732]
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 2

Logarithmic Forecast:
Step 70: 80.3164
Step 80: 81.4855
Step 90: 82.5167
Step 100: 83.4391
Step 110: 84.2736
Step 300: 93.0577
Step 330: 93.8921

Log-Linear Forecast:
Step 70: 79.8710
Step 80: 80.7648
Step 90: 81.4991
Step 100: 82.1075
Step 110: 82.6142
Step 300: 84.0602
Step 330: 83.6493

Power law Forecast:
Step 70: 80.0156
Step 80: 81.0443
Step 90: 81.9390
Step 100: 82.7292
Step 110: 83.4360
Step 300: 90.4317
Step 330: 91.0559

Mean Squared Error:
Logarithmic: 1.054310
Power law: 0.438590
Log linear: 0.363973


In [7]:
import numpy as np
from scipy import stats
from scipy.optimize import approx_fprime

def compute_confidence_band(best_func, x_fit, popt, pcov, steps, alpha=0.05):
    # Get predicted values
    y_fit = best_func(x_fit, *popt)
    n_params = len(popt)
    # Set small epsilon for numerical gradients
    eps = np.sqrt(np.finfo(float).eps)
    
    # Compute Jacobian: for each x in x_fit, estimate the gradient wrt parameters.
    # We assume best_func(x, *params) returns a scalar.
    J = np.zeros((len(x_fit), n_params))
    for i, x_val in enumerate(x_fit):
        def func(p):
            return best_func(x_val, *p)
        J[i, :] = approx_fprime(popt, func, eps)
    
    # Standard error for each predicted f(x)
    sigma_y = np.sqrt(np.sum(J.dot(pcov) * J, axis=1))
    
    # Degrees of freedom: number of data points minus the number of parameters
    dof = max(0, len(steps) - n_params)
    tval = stats.t.ppf(1 - alpha/2, dof)
    
    lower = y_fit - tval * sigma_y
    upper = y_fit + tval * sigma_y
    return lower, upper


In [19]:
best_func = power_func
# best_func = log_linear_func
# Extract step and bbox/AP columns
steps = df['epoch'].values
acc_scores = df['val/acc1'].values


# Fit the function
popt, pcov = curve_fit(best_func, steps[1:], acc_scores[1:], p0=[50, 50, 0.1], sigma=sigma[1:])
print(popt)
# popt, pcov = curve_fit(mixed_func, steps, acc_scores, p0=p0, maxfev=5000)

n_steps_per_10_epochs = 10
n_epoch_marks = [80, 90, 100]

MAX = n_epoch_marks[-1]/10 * n_steps_per_10_epochs
# Generate points for the fitted curve
x_fit = np.arange(steps[10], MAX, 1)
y_fit = best_func(x_fit, *popt)
# 
# # Calculate confidence intervals
# alpha = 0.7  # 95% confidence interval
# n = len(steps)
# p = len(popt)  # number of parameters
# dof = max(0, n - p)  # degrees of freedom
# 
# # Student's t-distribution for alpha/2 and dof
# tval = np.abs(stats.t.ppf(alpha / 2, dof))
# 
# # Calculate standard errors
# sigma = np.sqrt(np.diag(pcov))
# 
# # Calculate confidence intervals
# ci = tval * sigma
# 
# # Generate upper and lower confidence bounds
# lower = best_func(x_fit, *(popt - ci))
# upper = best_func(x_fit, *(popt + ci))
lower, upper = compute_confidence_band(best_func, x_fit, popt, pcov, steps, alpha=0.1)
# Create the plot
fig = go.Figure()

# Add the actual data points
fig.add_trace(go.Scatter(x=steps, y=acc_scores, mode="markers", name="Actual Data"))

# Add the fitted curve
fig.add_trace(go.Scatter(x=x_fit, y=y_fit, mode="lines", name="Power Law Fit"))


# Add the confidence intervals
fig.add_trace(
    go.Scatter(
        x=np.concatenate([x_fit, x_fit[::-1]]),
        y=np.concatenate([upper, lower[::-1]]),
        fill="toself",
        fillcolor="rgba(0,100,80,0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        hoverinfo="skip",
        showlegend=False,
    )
)

# Update layout
fig.update_layout(
    title="Image Classification Model Performance with Power Law Fit",
    xaxis_title="Steps",
    yaxis_title="Accuracy",
    legend_title="Legend",
    hovermode="x",
)
# plot one point corresponding to MAX
for i, mark in enumerate(n_epoch_marks):
    # y_MAX = best_func(MAX, *popt)
    # fig.add_trace(
    #     go.Scatter(
    #         x=[MAX],
    #         y=[y_MAX],
    #         mode="markers",
    #         name=f"100ep {y_MAX:.2f}",
    #         marker=dict(size=10, color="red"),
    #     )
    # )
    yy = best_func(mark * n_steps_per_10_epochs/10, *popt)
    xx = mark * n_steps_per_10_epochs/10
    fig.add_trace(
        go.Scatter(
            x=[xx],
            y=[yy],
            mode="markers",
            name=f"{mark}ep {yy:.2f}",
            marker=dict(size=10, color="red"),
        )
    )

# Show the plot
fig.show()

# Print the model parameters and their confidence intervals
print("Model Parameters:")
param_names = ["a", "b", "c"]
# for name, value, error in zip(param_names, popt, ci):
#     print(f"{name}: {value:.4f} ± {error:.4f}")

# Forecast for future steps
future_steps = np.array([90000, MAX])
forecasts = best_func(future_steps, *popt)

print("\nForecasts:")
for step, forecast in zip(future_steps, forecasts):
    print(f"Step {step}: {forecast:.4f}")

[1.48654000e+02 1.10978121e+02 1.13094271e-01]


Model Parameters:

Forecasts:
Step 90000.0: 118.1089
Step 100.0: 82.7292


In [22]:
# (Existing code above remains unchanged)

# Create forecasts from the fitted functions
future_steps = np.array([70, 80, 90, 100, 110, 300, 330])
log_forecast = log_func(future_steps, *popt_log)
power_forecast = power_func(future_steps, *popt_power)

fig = px.scatter(df, x="epoch", y="val/acc1")

# Update layout
fig.update_layout(
    title='Image Classification Model Performance with different model fits',
    xaxis_title='Epoch',
    yaxis_title='Accuracy',
    legend_title='Legend',
    hovermode="x"
)

# Add fitted curves
fig.add_scatter(x=x_fit, y=y_fit_log, mode='lines', name='Logarithmic fit')
fig.add_scatter(x=x_fit, y=y_fit_power, mode='lines', name='Power law fit')

# Add extra legend items and plot annotations for special epochs: 100, 200 and 300.
special_epochs = [100, 200, 300]
for epoch in special_epochs:
    power_pred = power_func(epoch, *popt_power)
    log_pred   = log_func(epoch, *popt_log)
    
    # Add a dummy marker that only appears in the legend with the forecast values
    fig.add_scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(color='red'),
        showlegend=True,
        name=f"Epoch {epoch}: Power={power_pred:.2f}, Log={log_pred:.2f}"
    )
    
    # Place an annotation on the plot at the midpoint between power and log predictions
    y_ann = (power_pred + log_pred) / 2
    fig.add_annotation(
        x=epoch,
        y=y_ann,
        text=f"[{power_pred:.2f}, {log_pred:.2f}]",
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-20
    )

fig.show()

# (The rest of your code such as printing forecasts and mse calculations remain unchanged)
